<a href="https://colab.research.google.com/github/neerajthandayan/Misc./blob/main/ResearchExecutiveTask/ResearchExecutiveTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np